In [3]:
%matplotlib inline

In [9]:
# -*- coding: utf-8 -*-
""" .....................

################################################################################
# Author: Weikun Han <weikunhan@gmail.com>
# Crate Date: 04/07/2018        
# Update:
# Reference: 
#    https://github.com/michalfaber/keras_Realtime_Multi-Person_Pose_Estimation
################################################################################
"""

import os
import cv2
import numpy as np
from pycocotools.coco import COCO

# Please modify input path  to locate you file
DATASETS_ROOT_DIR = './datasets'

# List sub-folder for datasets
mode = ['val', 'train']
index = 1

# Locate the datasets
OUTPUT_MASKS_DIR = os.path.join(
    DATASETS_ROOT_DIR, 'mask_{}_coco2014'.format(mode[index]))
annotations_path = os.path.join(
    DATASETS_ROOT_DIR, 
    'annotations_coco2014/person_keypoints_{}2014.json'.format(mode[index]))
images_path = os.path.join(
    DATASETS_ROOT_DIR, '{}_coco2014'.format(mode[index]))

# Check location to save datasets
if not os.path.exists(OUTPUT_MASKS_DIR):
    os.makedirs(OUTPUT_MASKS_DIR)

# Use COCO API to load data
coco = COCO(annotations_path)
img_ids = list(coco.imgs.keys())
for i, img_id in enumerate(img_ids):
    ann_ids = coco.getAnnIds(imgIds=img_id)
    img_anns = coco.loadAnns(ann_ids)

    img_filename = os.path.join(
        images_path, 'COCO_{}2014_{:012d}.jpg'.format(mode[index], img_id))
    mask_miss_path = os.path.join(
        OUTPUT_MASKS_DIR, 'mask_miss_{:012d}.png'.format(img_id))
    mask_all_path = os.path.join(
        OUTPUT_MASKS_DIR, 'mask_all_{:012d}.png'.format(img_id))
    
    # Use Open CV to read each picture
    img = cv2.imread(img_filename)
    high, width, channel = img.shape
    
    # initial each array to store mask
    mask_all = np.zeros((high, width), dtype=np.uint8)
    mask_miss = np.zeros((high, width), dtype=np.uint8)
    flag = 0
    
    for n in img_anns:
        seg = n["segmentation"]

        if n["iscrowd"] == 1:
            mask_crowd = coco.annToMask(n)
            temp = np.bitwise_and(mask_all, mask_crowd)
            mask_crowd = mask_crowd - temp
            flag += 1
            continue
        else:
            mask = coco.annToMask(n)

        mask_all = np.bitwise_or(mask, mask_all)

        if n["num_keypoints"] <= 0:
            mask_miss = np.bitwise_or(mask, mask_miss)

    if flag<1:
        mask_miss = np.logical_not(mask_miss)
    elif flag == 1:
        mask_miss = np.logical_not(np.bitwise_or(mask_miss, mask_crowd))
        mask_all = np.bitwise_or(mask_all, mask_crowd)
    else:
        raise Exception("crowd segments > 1")

    cv2.imwrite(mask_miss_path, mask_miss * 255)
    cv2.imwrite(mask_all_path, mask_all * 255)

    if (i % 1000 == 0):
        print("Processed %d of %d" % (i, len(img_ids)))

print("Done !!!")


loading annotations into memory...
Done (t=3.49s)
creating index...
index created!
Processed 0 of 82783
Processed 1000 of 82783
Processed 2000 of 82783
Processed 3000 of 82783
Processed 4000 of 82783
Processed 5000 of 82783
Processed 6000 of 82783
Processed 7000 of 82783
Processed 8000 of 82783
Processed 9000 of 82783
Processed 10000 of 82783
Processed 11000 of 82783
Processed 12000 of 82783
Processed 13000 of 82783
Processed 14000 of 82783
Processed 15000 of 82783
Processed 16000 of 82783
Processed 17000 of 82783
Processed 18000 of 82783
Processed 19000 of 82783
Processed 20000 of 82783
Processed 21000 of 82783
Processed 22000 of 82783
Processed 23000 of 82783
Processed 24000 of 82783
Processed 25000 of 82783
Processed 26000 of 82783
Processed 27000 of 82783
Processed 28000 of 82783
Processed 29000 of 82783
Processed 30000 of 82783
Processed 31000 of 82783
Processed 32000 of 82783
Processed 33000 of 82783
Processed 34000 of 82783
Processed 35000 of 82783
Processed 36000 of 82783
Proce